In [12]:
import torch
import numpy as np
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss

from util.data_preprocess import get_data_loader, calc_metrics
from model.CatModel import CatModel
from model.AddModel import AddModel
from model.AttentionCatModel import AttentionCatModel
from model.AttentionAddModel import AttentionAddModel

import argparse


def init_argparse():
    parser = argparse.ArgumentParser()
    parser.add_argument('--text_only', action='store_true', help='仅使用文本')
    parser.add_argument('--image_only', action='store_true', help='仅使用图片')
    parser.add_argument('--do_test', action='store_true', help='使用训练后的模型对测试集进行预测')
    parser.add_argument('--lr', default=5e-5, help='设置学习率', type=float)
    parser.add_argument('--weight_decay', default=1e-2, help='设置权重衰减', type=float)
    parser.add_argument('--epochs', default=10, help='设置训练轮数', type=int)
    parser.add_argument('--seed', default=233, help='设置随机种子', type=int)
    args = parser.parse_args()
    return args


# args = init_argparse()
# print('args:', args)
# """text_only和image_only互斥"""
# assert((args.text_only and args.image_only) == False)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

torch.manual_seed(1)
torch.cuda.manual_seed(1)
torch.cuda.manual_seed_all(1)
np.random.seed(1)
"""在种子不变的情况下保证结果一致"""
torch.backends.cudnn.deterministic = True

# args = {'lr': 1e-2, 'epochs': 5, 'weight_decay': 1e-5, 'text_only': False, 'image_only': False}
class Arg:
    def __init__(self):
        self.lr = 3e-5
        self.epochs = 5
        self.weight_decay = 1e-5
        self.text_only = False
        self.image_only = False
args = Arg()

def model_train():
    """训练模型并保存至./model.pth"""

    train_data_loader, valid_data_loader, test_data_loader = get_data_loader()
    model = AttentionCatModel.from_pretrained('./pre_trained/bert-base-uncased')
    model.to(device)
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': args.weight_decay},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    optimizer = AdamW(lr=args.lr, params=optimizer_grouped_parameters)
    criterion = CrossEntropyLoss()
    best_rate = 0
    print('[START_OF_TRAINING_STAGE]')
    for epoch in range(args.epochs):
        total_loss = 0
        correct = 0
        total = 0
        target_list = []
        pred_list = []
        model.train()
        for idx, (guid, tag, image, text) in enumerate(train_data_loader):
            tag = tag.to(device)
            image = image.to(device)
            text = text.to(device)
            # if args.text_only:
            #     out = model(image_input=None, text_input=text)
            # elif args.image_only:
            #     out = model(image_input=image, text_input=None)
            # else:
            #     out = model(image_input=image, text_input=text)
            out = model(image_input=image, text_input=text)
            loss = criterion(out, tag)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            total_loss += loss.item() * len(guid)
            pred = torch.max(out, 1)[1]
            total += len(guid)
            correct += (pred == tag).sum()

            target_list.extend(tag.cpu().tolist())
            pred_list.extend(pred.cpu().tolist())

        total_loss /= total
        print('[EPOCH{:02d}]'.format(epoch + 1), end='')
        print('[TRAIN] - LOSS:{:.6f}'.format(total_loss), end='')
        rate = correct / total * 100
        print(' ACC_RATE:{:.2f}%'.format(rate), end='')
        metrics = calc_metrics(target_list, pred_list)
        print(' WEIGHTED_ACC: {:.2f}% WEIGHTED_F1: {:.2f}% MAC_ACC: {:.2f}% MAC_F1: {:.2f}%'.format(metrics[0] * 100,
                                                                                                    metrics[2] * 100,
                                                                                                    metrics[3] * 100,
                                                                                                    metrics[5] * 100))

        total_loss = 0
        correct = 0
        total = 0
        target_list = []
        pred_list = []
        model.eval()

        for guid, tag, image, text in valid_data_loader:
            tag = tag.to(device)
            image = image.to(device)
            text = text.to(device)

            # if args.text_only:
            #     out = model(image_input=None, text_input=text)
            # elif args.image_only:
            #     out = model(image_input=image, text_input=None)
            # else:
            #     out = model(image_input=image, text_input=text)

            out = model(image_input=image, text_input=text)

            loss = criterion(out, tag)

            total_loss += loss.item() * len(guid)
            pred = torch.max(out, 1)[1]
            total += len(guid)
            correct += (pred == tag).sum()

            target_list.extend(tag.cpu().tolist())
            pred_list.extend(pred.cpu().tolist())

        total_loss /= total
        print('         [EVAL]  - LOSS:{:.6f}'.format(total_loss), end='')
        rate = correct / total * 100
        print(' ACC_RATE:{:.2f}%'.format(rate), end='')
        metrics = calc_metrics(target_list, pred_list)
        print(' WEIGHTED_ACC: {:.2f}% WEIGHTED_F1: {:.2f}% MAC_ACC: {:.2f}% MAC_F1: {:.2f}%'.format(metrics[0] * 100,
                                                                                                    metrics[2] * 100,
                                                                                                    metrics[3] * 100,
                                                                                                    metrics[5] * 100))

        if rate > best_rate:
            best_rate = rate
            print('         [SAVE] BEST ACC_RATE ON THE VALIDATION SET:{:.2f}%'.format(rate))
            torch.save(model.state_dict(), 'attention_cat_model.pth')
        print()
    print('[END_OF_TRAINING_STAGE]')

def model_test():
    """利用训练好的./model.pth对测试集进行预测，结果保存至output/test_with_label.txt"""

    train_data_list, test_data_list = get_data_list()
    train_data_list, test_data_list = data_preprocess(train_data_list, test_data_list)
    train_data_loader, valid_data_loader, test_data_loader = get_data_loader(train_data_list, test_data_list)
    model = MultimodalModel.from_pretrained('./pre_trained/bert-base-uncased')
    model.load_state_dict(torch.load('attention_cat_model.pth'))
    model.to(device)
    print('[START_OF_TESTING_STAGE]')
    guid_list = []
    pred_list = []
    model.eval()

    for guid, tag, image, text in test_data_loader:
        image = image.to(device)
        text = text.to(device)

        if args.text_only:
            out = model(image_input=None, text_input=text)
        elif args.image_only:
            out = model(image_input=image, text_input=None)
        else:
            out = model(image_input=image, text_input=text)

        pred = torch.max(out, 1)[1]
        guid_list.extend(guid)
        pred_list.extend(pred.cpu().tolist())

    pred_mapped = {
        0: 'negative',
        1: 'neutral',
        2: 'positive',
    }
    
    with open('output/attention_cat_model_test_with_label.txt', 'w', encoding='utf-8') as f:
        f.write('guid,tag\n')
        for guid, pred in zip(guid_list, pred_list):
            f.write(f'{guid},{pred_mapped[pred]}\n')
        f.close()
        print('[PREDICTION] SAVE TO output/test_with_label.txt')
    print('[END_OF_TESTING_STAGE]')

if __name__ == "__main__":
    # if args.do_test:
    #     model_test()
    # else:
    #     model_train()
    model_train()

Some weights of the model checkpoint at ./pre_trained/resnet-152 were not used when initializing ResNetModel: ['classifier.1.bias', 'classifier.1.weight']
- This IS expected if you are initializing ResNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ResNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at ./pre_trained/bert-base-uncased were not used when initializing AttentionCatModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.t

[START_OF_TRAINING_STAGE]
[EPOCH01][TRAIN] - LOSS:0.746794 ACC_RATE:67.22% WEIGHTED_ACC: 65.61% WEIGHTED_F1: 64.66% MAC_ACC: 61.68% MAC_F1: 51.37%
         [EVAL]  - LOSS:0.632219 ACC_RATE:75.50% WEIGHTED_ACC: 75.23% WEIGHTED_F1: 72.59% MAC_ACC: 73.30% MAC_F1: 57.54%
         [SAVE] BEST ACC_RATE ON THE VALIDATION SET:75.50%

[EPOCH02][TRAIN] - LOSS:0.461234 ACC_RATE:81.42% WEIGHTED_ACC: 80.75% WEIGHTED_F1: 80.69% MAC_ACC: 76.92% MAC_F1: 72.40%
         [EVAL]  - LOSS:0.846265 ACC_RATE:75.50% WEIGHTED_ACC: 74.22% WEIGHTED_F1: 73.25% MAC_ACC: 69.23% MAC_F1: 59.40%

[EPOCH03][TRAIN] - LOSS:0.259479 ACC_RATE:90.14% WEIGHTED_ACC: 89.96% WEIGHTED_F1: 90.00% MAC_ACC: 86.98% MAC_F1: 85.49%
         [EVAL]  - LOSS:0.825190 ACC_RATE:74.25% WEIGHTED_ACC: 74.37% WEIGHTED_F1: 74.28% MAC_ACC: 61.76% MAC_F1: 61.70%

[EPOCH04][TRAIN] - LOSS:0.130775 ACC_RATE:95.47% WEIGHTED_ACC: 95.45% WEIGHTED_F1: 95.46% MAC_ACC: 93.71% MAC_F1: 93.30%
         [EVAL]  - LOSS:0.953603 ACC_RATE:75.50% WEIGHTED_ACC: 75

In [11]:
import torch
import numpy as np
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss

from util.data_preprocess import get_data_loader, calc_metrics
from model.CatModel import CatModel
from model.AttentionCatModel import AttentionCatModel

import argparse


def init_argparse():
    parser = argparse.ArgumentParser()
    parser.add_argument('--text_only', action='store_true', help='仅使用文本')
    parser.add_argument('--image_only', action='store_true', help='仅使用图片')
    parser.add_argument('--do_test', action='store_true', help='使用训练后的模型对测试集进行预测')
    parser.add_argument('--lr', default=5e-5, help='设置学习率', type=float)
    parser.add_argument('--weight_decay', default=1e-2, help='设置权重衰减', type=float)
    parser.add_argument('--epochs', default=10, help='设置训练轮数', type=int)
    parser.add_argument('--seed', default=233, help='设置随机种子', type=int)
    args = parser.parse_args()
    return args


# args = init_argparse()
# print('args:', args)
# """text_only和image_only互斥"""
# assert((args.text_only and args.image_only) == False)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

torch.manual_seed(1)
torch.cuda.manual_seed(1)
torch.cuda.manual_seed_all(1)
np.random.seed(1)
"""在种子不变的情况下保证结果一致"""
torch.backends.cudnn.deterministic = True

# args = {'lr': 1e-2, 'epochs': 5, 'weight_decay': 1e-5, 'text_only': False, 'image_only': False}
class Arg:
    def __init__(self):
        self.lr = 3e-5
        self.epochs = 5
        self.weight_decay = 1e-5
        self.text_only = False
        self.image_only = False
args = Arg()

def model_train():
    """训练模型并保存至./model.pth"""

    train_data_loader, valid_data_loader, test_data_loader = get_data_loader()
    model = CatModel.from_pretrained('./pre_trained/bert-base-uncased')
    model.to(device)
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': args.weight_decay},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    optimizer = AdamW(lr=args.lr, params=optimizer_grouped_parameters)
    criterion = CrossEntropyLoss()
    best_rate = 0
    print('[START_OF_TRAINING_STAGE]')
    for epoch in range(args.epochs):
        total_loss = 0
        correct = 0
        total = 0
        target_list = []
        pred_list = []
        model.train()
        for idx, (guid, tag, image, text) in enumerate(train_data_loader):
            tag = tag.to(device)
            image = image.to(device)
            text = text.to(device)
            # if args.text_only:
            #     out = model(image_input=None, text_input=text)
            # elif args.image_only:
            #     out = model(image_input=image, text_input=None)
            # else:
            #     out = model(image_input=image, text_input=text)
            out = model(image_input=image, text_input=text)
            loss = criterion(out, tag)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            total_loss += loss.item() * len(guid)
            pred = torch.max(out, 1)[1]
            total += len(guid)
            correct += (pred == tag).sum()

            target_list.extend(tag.cpu().tolist())
            pred_list.extend(pred.cpu().tolist())

        total_loss /= total
        print('[EPOCH{:02d}]'.format(epoch + 1), end='')
        print('[TRAIN] - LOSS:{:.6f}'.format(total_loss), end='')
        rate = correct / total * 100
        print(' ACC_RATE:{:.2f}%'.format(rate), end='')
        metrics = calc_metrics(target_list, pred_list)
        print(' WEIGHTED_ACC: {:.2f}% WEIGHTED_F1: {:.2f}% MAC_ACC: {:.2f}% MAC_F1: {:.2f}%'.format(metrics[0] * 100,
                                                                                                    metrics[2] * 100,
                                                                                                    metrics[3] * 100,
                                                                                                    metrics[5] * 100))

        total_loss = 0
        correct = 0
        total = 0
        target_list = []
        pred_list = []
        model.eval()

        for guid, tag, image, text in valid_data_loader:
            tag = tag.to(device)
            image = image.to(device)
            text = text.to(device)

            # if args.text_only:
            #     out = model(image_input=None, text_input=text)
            # elif args.image_only:
            #     out = model(image_input=image, text_input=None)
            # else:
            #     out = model(image_input=image, text_input=text)

            out = model(image_input=image, text_input=text)

            loss = criterion(out, tag)

            total_loss += loss.item() * len(guid)
            pred = torch.max(out, 1)[1]
            total += len(guid)
            correct += (pred == tag).sum()

            target_list.extend(tag.cpu().tolist())
            pred_list.extend(pred.cpu().tolist())

        total_loss /= total
        print('         [EVAL]  - LOSS:{:.6f}'.format(total_loss), end='')
        rate = correct / total * 100
        print(' ACC_RATE:{:.2f}%'.format(rate), end='')
        metrics = calc_metrics(target_list, pred_list)
        print(' WEIGHTED_ACC: {:.2f}% WEIGHTED_F1: {:.2f}% MAC_ACC: {:.2f}% MAC_F1: {:.2f}%'.format(metrics[0] * 100,
                                                                                                    metrics[2] * 100,
                                                                                                    metrics[3] * 100,
                                                                                                    metrics[5] * 100))

        if rate > best_rate:
            best_rate = rate
            print('         [SAVE] BEST ACC_RATE ON THE VALIDATION SET:{:.2f}%'.format(rate))
            torch.save(model.state_dict(), 'cat_model.pth')
        print()
    print('[END_OF_TRAINING_STAGE]')

def model_test():
    """利用训练好的./model.pth对测试集进行预测，结果保存至output/test_with_label.txt"""

    train_data_list, test_data_list = get_data_list()
    train_data_list, test_data_list = data_preprocess(train_data_list, test_data_list)
    train_data_loader, valid_data_loader, test_data_loader = get_data_loader(train_data_list, test_data_list)
    model = MultimodalModel.from_pretrained('./pre_trained/bert-base-uncased')
    model.load_state_dict(torch.load('cat_model.pth'))
    model.to(device)
    print('[START_OF_TESTING_STAGE]')
    guid_list = []
    pred_list = []
    model.eval()

    for guid, tag, image, text in test_data_loader:
        image = image.to(device)
        text = text.to(device)

        if args.text_only:
            out = model(image_input=None, text_input=text)
        elif args.image_only:
            out = model(image_input=image, text_input=None)
        else:
            out = model(image_input=image, text_input=text)

        pred = torch.max(out, 1)[1]
        guid_list.extend(guid)
        pred_list.extend(pred.cpu().tolist())

    pred_mapped = {
        0: 'negative',
        1: 'neutral',
        2: 'positive',
    }
    
    with open('output/cat_model_test_with_label.txt', 'w', encoding='utf-8') as f:
        f.write('guid,tag\n')
        for guid, pred in zip(guid_list, pred_list):
            f.write(f'{guid},{pred_mapped[pred]}\n')
        f.close()
        print('[PREDICTION] SAVE TO output/test_with_label.txt')
    print('[END_OF_TESTING_STAGE]')

if __name__ == "__main__":
    # if args.do_test:
    #     model_test()
    # else:
    #     model_train()
    model_train()

Some weights of the model checkpoint at ./pre_trained/resnet-152 were not used when initializing ResNetModel: ['classifier.1.bias', 'classifier.1.weight']
- This IS expected if you are initializing ResNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ResNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at ./pre_trained/bert-base-uncased were not used when initializing CatModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.

[START_OF_TRAINING_STAGE]
[EPOCH01][TRAIN] - LOSS:0.809148 ACC_RATE:65.39% WEIGHTED_ACC: 63.00% WEIGHTED_F1: 63.00% MAC_ACC: 56.77% MAC_F1: 50.03%
         [EVAL]  - LOSS:0.702547 ACC_RATE:72.75% WEIGHTED_ACC: 69.38% WEIGHTED_F1: 69.33% MAC_ACC: 58.97% MAC_F1: 50.33%
         [SAVE] BEST ACC_RATE ON THE VALIDATION SET:72.75%

[EPOCH02][TRAIN] - LOSS:0.504848 ACC_RATE:79.25% WEIGHTED_ACC: 78.59% WEIGHTED_F1: 78.52% MAC_ACC: 74.56% MAC_F1: 70.08%
         [EVAL]  - LOSS:0.823378 ACC_RATE:72.25% WEIGHTED_ACC: 74.60% WEIGHTED_F1: 67.50% MAC_ACC: 81.42% MAC_F1: 47.28%

[EPOCH03][TRAIN] - LOSS:0.291790 ACC_RATE:89.33% WEIGHTED_ACC: 89.06% WEIGHTED_F1: 89.12% MAC_ACC: 85.62% MAC_F1: 83.67%
         [EVAL]  - LOSS:0.732769 ACC_RATE:75.50% WEIGHTED_ACC: 73.27% WEIGHTED_F1: 73.61% MAC_ACC: 62.23% MAC_F1: 56.50%
         [SAVE] BEST ACC_RATE ON THE VALIDATION SET:75.50%

[EPOCH04][TRAIN] - LOSS:0.164669 ACC_RATE:94.69% WEIGHTED_ACC: 94.65% WEIGHTED_F1: 94.65% MAC_ACC: 93.11% MAC_F1: 92.16%
      

In [10]:
import torch
import numpy as np
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss

from util.data_preprocess import get_data_loader, calc_metrics
from model.CatModel import CatModel
from model.AddModel import AddModel
from model.AttentionCatModel import AttentionCatModel

import argparse


def init_argparse():
    parser = argparse.ArgumentParser()
    parser.add_argument('--text_only', action='store_true', help='仅使用文本')
    parser.add_argument('--image_only', action='store_true', help='仅使用图片')
    parser.add_argument('--do_test', action='store_true', help='使用训练后的模型对测试集进行预测')
    parser.add_argument('--lr', default=5e-5, help='设置学习率', type=float)
    parser.add_argument('--weight_decay', default=1e-2, help='设置权重衰减', type=float)
    parser.add_argument('--epochs', default=10, help='设置训练轮数', type=int)
    parser.add_argument('--seed', default=233, help='设置随机种子', type=int)
    args = parser.parse_args()
    return args


# args = init_argparse()
# print('args:', args)
# """text_only和image_only互斥"""
# assert((args.text_only and args.image_only) == False)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

torch.manual_seed(1)
torch.cuda.manual_seed(1)
torch.cuda.manual_seed_all(1)
np.random.seed(1)
"""在种子不变的情况下保证结果一致"""
torch.backends.cudnn.deterministic = True

# args = {'lr': 1e-2, 'epochs': 5, 'weight_decay': 1e-5, 'text_only': False, 'image_only': False}
class Arg:
    def __init__(self):
        self.lr = 3e-5
        self.epochs = 5
        self.weight_decay = 1e-5
        self.text_only = False
        self.image_only = False
args = Arg()

def model_train():
    """训练模型并保存至./model.pth"""

    train_data_loader, valid_data_loader, test_data_loader = get_data_loader()
    model = AddModel.from_pretrained('./pre_trained/bert-base-uncased')
    model.to(device)
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': args.weight_decay},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    optimizer = AdamW(lr=args.lr, params=optimizer_grouped_parameters)
    criterion = CrossEntropyLoss()
    best_rate = 0
    print('[START_OF_TRAINING_STAGE]')
    for epoch in range(args.epochs):
        total_loss = 0
        correct = 0
        total = 0
        target_list = []
        pred_list = []
        model.train()
        for idx, (guid, tag, image, text) in enumerate(train_data_loader):
            tag = tag.to(device)
            image = image.to(device)
            text = text.to(device)
            # if args.text_only:
            #     out = model(image_input=None, text_input=text)
            # elif args.image_only:
            #     out = model(image_input=image, text_input=None)
            # else:
            #     out = model(image_input=image, text_input=text)
            out = model(image_input=image, text_input=text)
            loss = criterion(out, tag)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            total_loss += loss.item() * len(guid)
            pred = torch.max(out, 1)[1]
            total += len(guid)
            correct += (pred == tag).sum()

            target_list.extend(tag.cpu().tolist())
            pred_list.extend(pred.cpu().tolist())

        total_loss /= total
        print('[EPOCH{:02d}]'.format(epoch + 1), end='')
        print('[TRAIN] - LOSS:{:.6f}'.format(total_loss), end='')
        rate = correct / total * 100
        print(' ACC_RATE:{:.2f}%'.format(rate), end='')
        metrics = calc_metrics(target_list, pred_list)
        print(' WEIGHTED_ACC: {:.2f}% WEIGHTED_F1: {:.2f}% MAC_ACC: {:.2f}% MAC_F1: {:.2f}%'.format(metrics[0] * 100,
                                                                                                    metrics[2] * 100,
                                                                                                    metrics[3] * 100,
                                                                                                    metrics[5] * 100))

        total_loss = 0
        correct = 0
        total = 0
        target_list = []
        pred_list = []
        model.eval()

        for guid, tag, image, text in valid_data_loader:
            tag = tag.to(device)
            image = image.to(device)
            text = text.to(device)

            # if args.text_only:
            #     out = model(image_input=None, text_input=text)
            # elif args.image_only:
            #     out = model(image_input=image, text_input=None)
            # else:
            #     out = model(image_input=image, text_input=text)

            out = model(image_input=image, text_input=text)

            loss = criterion(out, tag)

            total_loss += loss.item() * len(guid)
            pred = torch.max(out, 1)[1]
            total += len(guid)
            correct += (pred == tag).sum()

            target_list.extend(tag.cpu().tolist())
            pred_list.extend(pred.cpu().tolist())

        total_loss /= total
        print('         [EVAL]  - LOSS:{:.6f}'.format(total_loss), end='')
        rate = correct / total * 100
        print(' ACC_RATE:{:.2f}%'.format(rate), end='')
        metrics = calc_metrics(target_list, pred_list)
        print(' WEIGHTED_ACC: {:.2f}% WEIGHTED_F1: {:.2f}% MAC_ACC: {:.2f}% MAC_F1: {:.2f}%'.format(metrics[0] * 100,
                                                                                                    metrics[2] * 100,
                                                                                                    metrics[3] * 100,
                                                                                                    metrics[5] * 100))

        if rate > best_rate:
            best_rate = rate
            print('         [SAVE] BEST ACC_RATE ON THE VALIDATION SET:{:.2f}%'.format(rate))
            torch.save(model.state_dict(), 'add_model.pth')
        print()
    print('[END_OF_TRAINING_STAGE]')

def model_test():
    """利用训练好的./model.pth对测试集进行预测，结果保存至output/test_with_label.txt"""

    train_data_list, test_data_list = get_data_list()
    train_data_list, test_data_list = data_preprocess(train_data_list, test_data_list)
    train_data_loader, valid_data_loader, test_data_loader = get_data_loader(train_data_list, test_data_list)
    model = AddModel.from_pretrained('./pre_trained/bert-base-uncased')
    model.load_state_dict(torch.load('add_model.pth'))
    model.to(device)
    print('[START_OF_TESTING_STAGE]')
    guid_list = []
    pred_list = []
    model.eval()

    for guid, tag, image, text in test_data_loader:
        image = image.to(device)
        text = text.to(device)

        if args.text_only:
            out = model(image_input=None, text_input=text)
        elif args.image_only:
            out = model(image_input=image, text_input=None)
        else:
            out = model(image_input=image, text_input=text)

        pred = torch.max(out, 1)[1]
        guid_list.extend(guid)
        pred_list.extend(pred.cpu().tolist())

    pred_mapped = {
        0: 'negative',
        1: 'neutral',
        2: 'positive',
    }
    
    with open('output/add_model_test_with_label.txt', 'w', encoding='utf-8') as f:
        f.write('guid,tag\n')
        for guid, pred in zip(guid_list, pred_list):
            f.write(f'{guid},{pred_mapped[pred]}\n')
        f.close()
        print('[PREDICTION] SAVE TO output/test_with_label.txt')
    print('[END_OF_TESTING_STAGE]')

if __name__ == "__main__":
    # if args.do_test:
    #     model_test()
    # else:
    #     model_train()
    model_train()

Some weights of the model checkpoint at ./pre_trained/resnet-152 were not used when initializing ResNetModel: ['classifier.1.bias', 'classifier.1.weight']
- This IS expected if you are initializing ResNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ResNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at ./pre_trained/bert-base-uncased were not used when initializing AddModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.

[START_OF_TRAINING_STAGE]
[EPOCH01][TRAIN] - LOSS:0.841869 ACC_RATE:63.28% WEIGHTED_ACC: 60.74% WEIGHTED_F1: 60.91% MAC_ACC: 54.23% MAC_F1: 48.13%
         [EVAL]  - LOSS:0.687330 ACC_RATE:70.75% WEIGHTED_ACC: 69.05% WEIGHTED_F1: 69.54% MAC_ACC: 54.91% MAC_F1: 52.91%
         [SAVE] BEST ACC_RATE ON THE VALIDATION SET:70.75%

[EPOCH02][TRAIN] - LOSS:0.530456 ACC_RATE:78.39% WEIGHTED_ACC: 77.69% WEIGHTED_F1: 77.48% MAC_ACC: 73.85% MAC_F1: 68.34%
         [EVAL]  - LOSS:0.667649 ACC_RATE:74.25% WEIGHTED_ACC: 72.80% WEIGHTED_F1: 73.24% MAC_ACC: 63.18% MAC_F1: 60.19%
         [SAVE] BEST ACC_RATE ON THE VALIDATION SET:74.25%

[EPOCH03][TRAIN] - LOSS:0.312937 ACC_RATE:88.14% WEIGHTED_ACC: 87.88% WEIGHTED_F1: 87.89% MAC_ACC: 84.97% MAC_F1: 82.50%
         [EVAL]  - LOSS:0.715693 ACC_RATE:75.25% WEIGHTED_ACC: 73.79% WEIGHTED_F1: 74.21% MAC_ACC: 64.51% MAC_F1: 61.17%
         [SAVE] BEST ACC_RATE ON THE VALIDATION SET:75.25%

[EPOCH04][TRAIN] - LOSS:0.177109 ACC_RATE:94.36% WEIGHTED_ACC: 94.30

In [5]:
import torch
import numpy as np
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss

from util.data_preprocess import get_data_loader, calc_metrics
from model.CatModel import CatModel
from model.AddModel import AddModel
from model.AttentionCatModel import AttentionCatModel
from model.AttentionAddModel import AttentionAddModel

import argparse


def init_argparse():
    parser = argparse.ArgumentParser()
    parser.add_argument('--text_only', action='store_true', help='仅使用文本')
    parser.add_argument('--image_only', action='store_true', help='仅使用图片')
    parser.add_argument('--do_test', action='store_true', help='使用训练后的模型对测试集进行预测')
    parser.add_argument('--lr', default=5e-5, help='设置学习率', type=float)
    parser.add_argument('--weight_decay', default=1e-2, help='设置权重衰减', type=float)
    parser.add_argument('--epochs', default=10, help='设置训练轮数', type=int)
    parser.add_argument('--seed', default=233, help='设置随机种子', type=int)
    args = parser.parse_args()
    return args


# args = init_argparse()
# print('args:', args)
# """text_only和image_only互斥"""
# assert((args.text_only and args.image_only) == False)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

torch.manual_seed(1)
torch.cuda.manual_seed(1)
torch.cuda.manual_seed_all(1)
np.random.seed(1)
"""在种子不变的情况下保证结果一致"""
torch.backends.cudnn.deterministic = True

# args = {'lr': 1e-2, 'epochs': 5, 'weight_decay': 1e-5, 'text_only': False, 'image_only': False}
class Arg:
    def __init__(self):
        self.lr = 3e-5
        self.epochs = 5
        self.weight_decay = 1e-5
        self.text_only = False
        self.image_only = False
args = Arg()

def model_train():
    """训练模型并保存至./model.pth"""

    train_data_loader, valid_data_loader, test_data_loader = get_data_loader()
    model = AttentionAddModel.from_pretrained('./pre_trained/bert-base-uncased')
    model.to(device)
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': args.weight_decay},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    optimizer = AdamW(lr=args.lr, params=optimizer_grouped_parameters)
    criterion = CrossEntropyLoss()
    best_rate = 0
    print('[START_OF_TRAINING_STAGE]')
    for epoch in range(args.epochs):
        total_loss = 0
        correct = 0
        total = 0
        target_list = []
        pred_list = []
        model.train()
        for idx, (guid, tag, image, text) in enumerate(train_data_loader):
            tag = tag.to(device)
            image = image.to(device)
            text = text.to(device)
            # if args.text_only:
            #     out = model(image_input=None, text_input=text)
            # elif args.image_only:
            #     out = model(image_input=image, text_input=None)
            # else:
            #     out = model(image_input=image, text_input=text)
            out = model(image_input=image, text_input=text)
            loss = criterion(out, tag)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            total_loss += loss.item() * len(guid)
            pred = torch.max(out, 1)[1]
            total += len(guid)
            correct += (pred == tag).sum()

            target_list.extend(tag.cpu().tolist())
            pred_list.extend(pred.cpu().tolist())

        total_loss /= total
        print('[EPOCH{:02d}]'.format(epoch + 1), end='')
        print('[TRAIN] - LOSS:{:.6f}'.format(total_loss), end='')
        rate = correct / total * 100
        print(' ACC_RATE:{:.2f}%'.format(rate), end='')
        metrics = calc_metrics(target_list, pred_list)
        print(' WEIGHTED_ACC: {:.2f}% WEIGHTED_F1: {:.2f}% MAC_ACC: {:.2f}% MAC_F1: {:.2f}%'.format(metrics[0] * 100,
                                                                                                    metrics[2] * 100,
                                                                                                    metrics[3] * 100,
                                                                                                    metrics[5] * 100))

        total_loss = 0
        correct = 0
        total = 0
        target_list = []
        pred_list = []
        model.eval()

        for guid, tag, image, text in valid_data_loader:
            tag = tag.to(device)
            image = image.to(device)
            text = text.to(device)

            # if args.text_only:
            #     out = model(image_input=None, text_input=text)
            # elif args.image_only:
            #     out = model(image_input=image, text_input=None)
            # else:
            #     out = model(image_input=image, text_input=text)

            out = model(image_input=image, text_input=text)

            loss = criterion(out, tag)

            total_loss += loss.item() * len(guid)
            pred = torch.max(out, 1)[1]
            total += len(guid)
            correct += (pred == tag).sum()

            target_list.extend(tag.cpu().tolist())
            pred_list.extend(pred.cpu().tolist())

        total_loss /= total
        print('         [EVAL]  - LOSS:{:.6f}'.format(total_loss), end='')
        rate = correct / total * 100
        print(' ACC_RATE:{:.2f}%'.format(rate), end='')
        metrics = calc_metrics(target_list, pred_list)
        print(' WEIGHTED_ACC: {:.2f}% WEIGHTED_F1: {:.2f}% MAC_ACC: {:.2f}% MAC_F1: {:.2f}%'.format(metrics[0] * 100,
                                                                                                    metrics[2] * 100,
                                                                                                    metrics[3] * 100,
                                                                                                    metrics[5] * 100))

        if rate > best_rate:
            best_rate = rate
            print('         [SAVE] BEST ACC_RATE ON THE VALIDATION SET:{:.2f}%'.format(rate))
            torch.save(model.state_dict(), 'model.pth')
        print()
    print('[END_OF_TRAINING_STAGE]')

def model_test():
    """利用训练好的./model.pth对测试集进行预测，结果保存至output/test_with_label.txt"""

    train_data_list, test_data_list = get_data_list()
    train_data_list, test_data_list = data_preprocess(train_data_list, test_data_list)
    train_data_loader, valid_data_loader, test_data_loader = get_data_loader(train_data_list, test_data_list)
    model = MultimodalModel.from_pretrained('bert-base-uncased')
    model.load_state_dict(torch.load('model.pth'))
    model.to(device)
    print('[START_OF_TESTING_STAGE]')
    guid_list = []
    pred_list = []
    model.eval()

    for guid, tag, image, text in test_data_loader:
        image = image.to(device)
        text = text.to(device)

        if args.text_only:
            out = model(image_input=None, text_input=text)
        elif args.image_only:
            out = model(image_input=image, text_input=None)
        else:
            out = model(image_input=image, text_input=text)

        pred = torch.max(out, 1)[1]
        guid_list.extend(guid)
        pred_list.extend(pred.cpu().tolist())

    pred_mapped = {
        0: 'negative',
        1: 'neutral',
        2: 'positive',
    }
    
    with open('output/test_with_label.txt', 'w', encoding='utf-8') as f:
        f.write('guid,tag\n')
        for guid, pred in zip(guid_list, pred_list):
            f.write(f'{guid},{pred_mapped[pred]}\n')
        f.close()
        print('[PREDICTION] SAVE TO output/test_with_label.txt')
    print('[END_OF_TESTING_STAGE]')

if __name__ == "__main__":
    # if args.do_test:
    #     model_test()
    # else:
    #     model_train()
    model_train()

Some weights of the model checkpoint at ./pre_trained/resnet-152 were not used when initializing ResNetModel: ['classifier.1.bias', 'classifier.1.weight']
- This IS expected if you are initializing ResNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ResNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at ./pre_trained/bert-base-uncased were not used when initializing AttentionAddModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.t

[START_OF_TRAINING_STAGE]
[EPOCH01][TRAIN] - LOSS:0.734719 ACC_RATE:68.67% WEIGHTED_ACC: 67.03% WEIGHTED_F1: 66.21% MAC_ACC: 63.00% MAC_F1: 53.19%
         [EVAL]  - LOSS:0.639342 ACC_RATE:75.00% WEIGHTED_ACC: 74.00% WEIGHTED_F1: 72.90% MAC_ACC: 70.78% MAC_F1: 58.17%
         [SAVE] BEST ACC_RATE ON THE VALIDATION SET:75.00%

[EPOCH02][TRAIN] - LOSS:0.481952 ACC_RATE:80.36% WEIGHTED_ACC: 79.64% WEIGHTED_F1: 79.65% MAC_ACC: 74.81% MAC_F1: 70.86%
         [EVAL]  - LOSS:0.745884 ACC_RATE:70.00% WEIGHTED_ACC: 72.48% WEIGHTED_F1: 70.40% MAC_ACC: 59.87% MAC_F1: 59.79%

[EPOCH03][TRAIN] - LOSS:0.248432 ACC_RATE:90.31% WEIGHTED_ACC: 90.15% WEIGHTED_F1: 90.19% MAC_ACC: 87.02% MAC_F1: 85.67%
         [EVAL]  - LOSS:0.856114 ACC_RATE:70.75% WEIGHTED_ACC: 73.05% WEIGHTED_F1: 71.61% MAC_ACC: 59.94% MAC_F1: 61.30%

[EPOCH04][TRAIN] - LOSS:0.154680 ACC_RATE:94.67% WEIGHTED_ACC: 94.63% WEIGHTED_F1: 94.64% MAC_ACC: 93.13% MAC_F1: 92.36%
         [EVAL]  - LOSS:0.988019 ACC_RATE:74.75% WEIGHTED_ACC: 72

In [9]:
def model_test():
    """利用训练好的./model.pth对测试集进行预测，结果保存至output/test_with_label.txt"""
    train_data_loader, valid_data_loader, test_data_loader = get_data_loader()
    model = AttentionAddModel.from_pretrained('./pre_trained/bert-base-uncased')
    model.load_state_dict(torch.load('model.pth'))
    model.to(device)
    print('[START_OF_TESTING_STAGE]')
    guid_list = []
    pred_list = []
    model.eval()

    for guid, tag, image, text in test_data_loader:
        image = image.to(device)
        text = text.to(device)

        if args.text_only:
            out = model(image_input=None, text_input=text)
        elif args.image_only:
            out = model(image_input=image, text_input=None)
        else:
            out = model(image_input=image, text_input=text)

        pred = torch.max(out, 1)[1]
        guid_list.extend(guid)
        pred_list.extend(pred.cpu().tolist())

    pred_mapped = {
        0: 'negative',
        1: 'neutral',
        2: 'positive',
    }
    
    with open('output/test_with_label.txt', 'w', encoding='utf-8') as f:
        f.write('guid,tag\n')
        for guid, pred in zip(guid_list, pred_list):
            f.write(f'{guid},{pred_mapped[pred]}\n')
        f.close()
        print('[PREDICTION] SAVE TO output/test_with_label.txt')
    print('[END_OF_TESTING_STAGE]')

model_test()

Some weights of the model checkpoint at ./pre_trained/resnet-152 were not used when initializing ResNetModel: ['classifier.1.bias', 'classifier.1.weight']
- This IS expected if you are initializing ResNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ResNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at ./pre_trained/bert-base-uncased were not used when initializing AttentionAddModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.t

[START_OF_TESTING_STAGE]
[PREDICTION] SAVE TO output/test_with_label.txt
[END_OF_TESTING_STAGE]


In [15]:
def model_test(model_name):
    """利用训练好的./model.pth对测试集进行预测，结果保存至output/test_with_label.txt"""
    train_data_loader, valid_data_loader, test_data_loader = get_data_loader()
    if model_name == 'attention_cat_model':
        model = AttentionCatModel.from_pretrained('./pre_trained/bert-base-uncased')
    elif model_name == 'cat_model':
        model = CatModel.from_pretrained('./pre_trained/bert-base-uncased')
    elif model_name == 'add_model':
        model = AddModel.from_pretrained('./pre_trained/bert-base-uncased')
    model.load_state_dict(torch.load(model_name + '.pth'))
    model.to(device)
    print('[START_OF_TESTING_STAGE]')
    guid_list = []
    pred_list = []
    model.eval()

    for guid, tag, image, text in test_data_loader:
        image = image.to(device)
        text = text.to(device)

        if args.text_only:
            out = model(image_input=None, text_input=text)
        elif args.image_only:
            out = model(image_input=image, text_input=None)
        else:
            out = model(image_input=image, text_input=text)

        pred = torch.max(out, 1)[1]
        guid_list.extend(guid)
        pred_list.extend(pred.cpu().tolist())

    pred_mapped = {
        0: 'negative',
        1: 'neutral',
        2: 'positive',
    }
    
    with open('output/' + model_name + '_test_with_label.txt', 'w', encoding='utf-8') as f:
        f.write('guid,tag\n')
        for guid, pred in zip(guid_list, pred_list):
            f.write(f'{guid},{pred_mapped[pred]}\n')
        f.close()
        print('[PREDICTION] SAVE TO output/test_with_label.txt')
    print('[END_OF_TESTING_STAGE]')
    
model_test('attention_cat_model')
model_test('cat_model')
model_test('add_model')

Some weights of the model checkpoint at ./pre_trained/resnet-152 were not used when initializing ResNetModel: ['classifier.1.bias', 'classifier.1.weight']
- This IS expected if you are initializing ResNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ResNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at ./pre_trained/bert-base-uncased were not used when initializing AttentionCatModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.t

[START_OF_TESTING_STAGE]
[PREDICTION] SAVE TO output/test_with_label.txt
[END_OF_TESTING_STAGE]


Some weights of the model checkpoint at ./pre_trained/resnet-152 were not used when initializing ResNetModel: ['classifier.1.bias', 'classifier.1.weight']
- This IS expected if you are initializing ResNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ResNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at ./pre_trained/bert-base-uncased were not used when initializing CatModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.

[START_OF_TESTING_STAGE]
[PREDICTION] SAVE TO output/test_with_label.txt
[END_OF_TESTING_STAGE]


Some weights of the model checkpoint at ./pre_trained/resnet-152 were not used when initializing ResNetModel: ['classifier.1.bias', 'classifier.1.weight']
- This IS expected if you are initializing ResNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ResNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at ./pre_trained/bert-base-uncased were not used when initializing AddModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.

[START_OF_TESTING_STAGE]
[PREDICTION] SAVE TO output/test_with_label.txt
[END_OF_TESTING_STAGE]


In [21]:
class Arg:
    def __init__(self):
        self.lr = 3e-5
        self.epochs = 5
        self.weight_decay = 1e-5
        self.text_only = True
        self.image_only = False
args = Arg()

def model_train():
    """训练模型并保存至./model.pth"""

    train_data_loader, valid_data_loader, test_data_loader = get_data_loader()
    model = AttentionCatModel.from_pretrained('./pre_trained/bert-base-uncased')
    model.to(device)
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': args.weight_decay},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    optimizer = AdamW(lr=args.lr, params=optimizer_grouped_parameters)
    criterion = CrossEntropyLoss()
    best_rate = 0
    print('[START_OF_TRAINING_STAGE]')
    for epoch in range(args.epochs):
        total_loss = 0
        correct = 0
        total = 0
        target_list = []
        pred_list = []
        model.train()
        for idx, (guid, tag, image, text) in enumerate(train_data_loader):
            tag = tag.to(device)
            image = image.to(device)
            text = text.to(device)
            if args.text_only:
                out = model(image_input=None, text_input=text)
            elif args.image_only:
                out = model(image_input=image, text_input=None)
            else:
                out = model(image_input=image, text_input=text)
            loss = criterion(out, tag)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            total_loss += loss.item() * len(guid)
            pred = torch.max(out, 1)[1]
            total += len(guid)
            correct += (pred == tag).sum()

            target_list.extend(tag.cpu().tolist())
            pred_list.extend(pred.cpu().tolist())

        total_loss /= total
        print('[EPOCH{:02d}]'.format(epoch + 1), end='')
        print('[TRAIN] - LOSS:{:.6f}'.format(total_loss), end='')
        rate = correct / total * 100
        print(' ACC_RATE:{:.2f}%'.format(rate), end='')
        metrics = calc_metrics(target_list, pred_list)
        print(' WEIGHTED_ACC: {:.2f}% WEIGHTED_F1: {:.2f}% MAC_ACC: {:.2f}% MAC_F1: {:.2f}%'.format(metrics[0] * 100,
                                                                                                    metrics[2] * 100,
                                                                                                    metrics[3] * 100,
                                                                                                    metrics[5] * 100))

        total_loss = 0
        correct = 0
        total = 0
        target_list = []
        pred_list = []
        model.eval()

        for guid, tag, image, text in valid_data_loader:
            tag = tag.to(device)
            image = image.to(device)
            text = text.to(device)

            if args.text_only:
                out = model(image_input=None, text_input=text)
            elif args.image_only:
                out = model(image_input=image, text_input=None)
            else:
                out = model(image_input=image, text_input=text)

            loss = criterion(out, tag)

            total_loss += loss.item() * len(guid)
            pred = torch.max(out, 1)[1]
            total += len(guid)
            correct += (pred == tag).sum()

            target_list.extend(tag.cpu().tolist())
            pred_list.extend(pred.cpu().tolist())

        total_loss /= total
        print('         [EVAL]  - LOSS:{:.6f}'.format(total_loss), end='')
        rate = correct / total * 100
        print(' ACC_RATE:{:.2f}%'.format(rate), end='')
        metrics = calc_metrics(target_list, pred_list)
        print(' WEIGHTED_ACC: {:.2f}% WEIGHTED_F1: {:.2f}% MAC_ACC: {:.2f}% MAC_F1: {:.2f}%'.format(metrics[0] * 100,
                                                                                                    metrics[2] * 100,
                                                                                                    metrics[3] * 100,
                                                                                                    metrics[5] * 100))
        print()
    print('[END_OF_TRAINING_STAGE]')
    
model_train()

Some weights of the model checkpoint at ./pre_trained/resnet-152 were not used when initializing ResNetModel: ['classifier.1.bias', 'classifier.1.weight']
- This IS expected if you are initializing ResNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ResNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at ./pre_trained/bert-base-uncased were not used when initializing AttentionCatModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.t

[START_OF_TRAINING_STAGE]
[EPOCH01][TRAIN] - LOSS:0.758277 ACC_RATE:67.42% WEIGHTED_ACC: 66.73% WEIGHTED_F1: 63.85% MAC_ACC: 66.24% MAC_F1: 49.04%
         [EVAL]  - LOSS:0.659584 ACC_RATE:71.00% WEIGHTED_ACC: 70.39% WEIGHTED_F1: 69.13% MAC_ACC: 68.17% MAC_F1: 58.64%

[EPOCH02][TRAIN] - LOSS:0.541316 ACC_RATE:78.92% WEIGHTED_ACC: 77.53% WEIGHTED_F1: 77.45% MAC_ACC: 71.93% MAC_F1: 65.73%
         [EVAL]  - LOSS:0.674514 ACC_RATE:70.50% WEIGHTED_ACC: 71.82% WEIGHTED_F1: 70.64% MAC_ACC: 63.97% MAC_F1: 62.54%

[EPOCH03][TRAIN] - LOSS:0.330589 ACC_RATE:87.69% WEIGHTED_ACC: 87.40% WEIGHTED_F1: 87.48% MAC_ACC: 83.12% MAC_F1: 81.48%
         [EVAL]  - LOSS:0.822170 ACC_RATE:70.50% WEIGHTED_ACC: 69.19% WEIGHTED_F1: 69.56% MAC_ACC: 61.32% MAC_F1: 58.53%

[EPOCH04][TRAIN] - LOSS:0.179525 ACC_RATE:93.81% WEIGHTED_ACC: 93.76% WEIGHTED_F1: 93.77% MAC_ACC: 92.09% MAC_F1: 91.19%
         [EVAL]  - LOSS:1.130940 ACC_RATE:65.50% WEIGHTED_ACC: 68.28% WEIGHTED_F1: 66.53% MAC_ACC: 56.51% MAC_F1: 57.10%

[E

In [4]:
class Arg:
    def __init__(self):
        self.lr = 3e-5
        self.epochs = 5
        self.weight_decay = 1e-5
        self.text_only = False
        self.image_only = True
args = Arg()

def model_train():
    """训练模型并保存至./model.pth"""

    train_data_loader, valid_data_loader, test_data_loader = get_data_loader()
    model = AttentionCatModel.from_pretrained('./pre_trained/bert-base-uncased')
    model.to(device)
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': args.weight_decay},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    optimizer = AdamW(lr=args.lr, params=optimizer_grouped_parameters)
    criterion = CrossEntropyLoss()
    best_rate = 0
    print('[START_OF_TRAINING_STAGE]')
    for epoch in range(args.epochs):
        total_loss = 0
        correct = 0
        total = 0
        target_list = []
        pred_list = []
        model.train()
        for idx, (guid, tag, image, text) in enumerate(train_data_loader):
            tag = tag.to(device)
            image = image.to(device)
            text = text.to(device)
            if args.text_only:
                out = model(image_input=None, text_input=text)
            elif args.image_only:
                out = model(image_input=image, text_input=None)
            else:
                out = model(image_input=image, text_input=text)
            loss = criterion(out, tag)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            total_loss += loss.item() * len(guid)
            pred = torch.max(out, 1)[1]
            total += len(guid)
            correct += (pred == tag).sum()

            target_list.extend(tag.cpu().tolist())
            pred_list.extend(pred.cpu().tolist())

        total_loss /= total
        print('[EPOCH{:02d}]'.format(epoch + 1), end='')
        print('[TRAIN] - LOSS:{:.6f}'.format(total_loss), end='')
        rate = correct / total * 100
        print(' ACC_RATE:{:.2f}%'.format(rate), end='')
        metrics = calc_metrics(target_list, pred_list)
        print(' WEIGHTED_ACC: {:.2f}% WEIGHTED_F1: {:.2f}% MAC_ACC: {:.2f}% MAC_F1: {:.2f}%'.format(metrics[0] * 100,
                                                                                                    metrics[2] * 100,
                                                                                                    metrics[3] * 100,
                                                                                                    metrics[5] * 100))

        total_loss = 0
        correct = 0
        total = 0
        target_list = []
        pred_list = []
        model.eval()

        for guid, tag, image, text in valid_data_loader:
            tag = tag.to(device)
            image = image.to(device)
            text = text.to(device)

            if args.text_only:
                out = model(image_input=None, text_input=text)
            elif args.image_only:
                out = model(image_input=image, text_input=None)
            else:
                out = model(image_input=image, text_input=text)

            loss = criterion(out, tag)

            total_loss += loss.item() * len(guid)
            pred = torch.max(out, 1)[1]
            total += len(guid)
            correct += (pred == tag).sum()

            target_list.extend(tag.cpu().tolist())
            pred_list.extend(pred.cpu().tolist())

        total_loss /= total
        print('         [EVAL]  - LOSS:{:.6f}'.format(total_loss), end='')
        rate = correct / total * 100
        print(' ACC_RATE:{:.2f}%'.format(rate), end='')
        metrics = calc_metrics(target_list, pred_list)
        print(' WEIGHTED_ACC: {:.2f}% WEIGHTED_F1: {:.2f}% MAC_ACC: {:.2f}% MAC_F1: {:.2f}%'.format(metrics[0] * 100,
                                                                                                    metrics[2] * 100,
                                                                                                    metrics[3] * 100,
                                                                                                    metrics[5] * 100))
        print()
    print('[END_OF_TRAINING_STAGE]')
    
model_train()

Some weights of the model checkpoint at ./pre_trained/resnet-152 were not used when initializing ResNetModel: ['classifier.1.weight', 'classifier.1.bias']
- This IS expected if you are initializing ResNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ResNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at ./pre_trained/bert-base-uncased were not used when initializing AttentionCatModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.pre

[START_OF_TRAINING_STAGE]
[EPOCH01][TRAIN] - LOSS:0.865595 ACC_RATE:61.17% WEIGHTED_ACC: 56.07% WEIGHTED_F1: 54.00% MAC_ACC: 49.34% MAC_F1: 35.74%
         [EVAL]  - LOSS:0.883641 ACC_RATE:57.50% WEIGHTED_ACC: 53.05% WEIGHTED_F1: 43.99% MAC_ACC: 40.16% MAC_F1: 26.80%

[EPOCH02][TRAIN] - LOSS:0.748008 ACC_RATE:66.22% WEIGHTED_ACC: 63.64% WEIGHTED_F1: 63.33% MAC_ACC: 57.73% MAC_F1: 50.08%
         [EVAL]  - LOSS:0.833191 ACC_RATE:60.00% WEIGHTED_ACC: 57.23% WEIGHTED_F1: 57.35% MAC_ACC: 49.36% MAC_F1: 47.45%

[EPOCH03][TRAIN] - LOSS:0.637249 ACC_RATE:73.19% WEIGHTED_ACC: 72.19% WEIGHTED_F1: 72.03% MAC_ACC: 68.44% MAC_F1: 62.59%
         [EVAL]  - LOSS:0.892197 ACC_RATE:62.25% WEIGHTED_ACC: 59.31% WEIGHTED_F1: 58.87% MAC_ACC: 52.03% MAC_F1: 48.51%

[EPOCH04][TRAIN] - LOSS:0.529114 ACC_RATE:78.19% WEIGHTED_ACC: 77.50% WEIGHTED_F1: 77.57% MAC_ACC: 73.24% MAC_F1: 69.69%
         [EVAL]  - LOSS:0.911858 ACC_RATE:58.75% WEIGHTED_ACC: 58.99% WEIGHTED_F1: 58.68% MAC_ACC: 49.53% MAC_F1: 48.55%

[E

In [1]:
import torch
import numpy as np
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss

from util.data_preprocess import get_data_loader, calc_metrics
from model.CatModel import CatModel
from model.AddModel import AddModel
from model.AttentionCatModel import AttentionCatModel
from model.AttentionAddModel import AttentionAddModel

import argparse

class Arg:
    def __init__(self):
        self.lr = 3e-5
        self.epochs = 5
        self.weight_decay = 1e-5
        self.text_only = False
        self.image_only = True
args = Arg()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

torch.manual_seed(1)
torch.cuda.manual_seed(1)
torch.cuda.manual_seed_all(1)
np.random.seed(1)
"""在种子不变的情况下保证结果一致"""
torch.backends.cudnn.deterministic = True

def model_train():
    """训练模型并保存至./model.pth"""

    train_data_loader, valid_data_loader, test_data_loader = get_data_loader()
    model = CatModel.from_pretrained('./pre_trained/bert-base-uncased')
    model.to(device)
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': args.weight_decay},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    optimizer = AdamW(lr=args.lr, params=optimizer_grouped_parameters)
    criterion = CrossEntropyLoss()
    best_rate = 0
    print('[START_OF_TRAINING_STAGE]')
    for epoch in range(args.epochs):
        total_loss = 0
        correct = 0
        total = 0
        target_list = []
        pred_list = []
        model.train()
        for idx, (guid, tag, image, text) in enumerate(train_data_loader):
            tag = tag.to(device)
            image = image.to(device)
            text = text.to(device)
            if args.text_only:
                out = model(image_input=None, text_input=text)
            elif args.image_only:
                out = model(image_input=image, text_input=None)
            else:
                out = model(image_input=image, text_input=text)
            loss = criterion(out, tag)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            total_loss += loss.item() * len(guid)
            pred = torch.max(out, 1)[1]
            total += len(guid)
            correct += (pred == tag).sum()

            target_list.extend(tag.cpu().tolist())
            pred_list.extend(pred.cpu().tolist())

        total_loss /= total
        print('[EPOCH{:02d}]'.format(epoch + 1), end='')
        print('[TRAIN] - LOSS:{:.6f}'.format(total_loss), end='')
        rate = correct / total * 100
        print(' ACC_RATE:{:.2f}%'.format(rate), end='')
        metrics = calc_metrics(target_list, pred_list)
        print(' WEIGHTED_ACC: {:.2f}% WEIGHTED_F1: {:.2f}% MAC_ACC: {:.2f}% MAC_F1: {:.2f}%'.format(metrics[0] * 100,
                                                                                                    metrics[2] * 100,
                                                                                                    metrics[3] * 100,
                                                                                                    metrics[5] * 100))

        total_loss = 0
        correct = 0
        total = 0
        target_list = []
        pred_list = []
        model.eval()

        for guid, tag, image, text in valid_data_loader:
            tag = tag.to(device)
            image = image.to(device)
            text = text.to(device)

            if args.text_only:
                out = model(image_input=None, text_input=text)
            elif args.image_only:
                out = model(image_input=image, text_input=None)
            else:
                out = model(image_input=image, text_input=text)

            loss = criterion(out, tag)

            total_loss += loss.item() * len(guid)
            pred = torch.max(out, 1)[1]
            total += len(guid)
            correct += (pred == tag).sum()

            target_list.extend(tag.cpu().tolist())
            pred_list.extend(pred.cpu().tolist())

        total_loss /= total
        print('         [EVAL]  - LOSS:{:.6f}'.format(total_loss), end='')
        rate = correct / total * 100
        print(' ACC_RATE:{:.2f}%'.format(rate), end='')
        metrics = calc_metrics(target_list, pred_list)
        print(' WEIGHTED_ACC: {:.2f}% WEIGHTED_F1: {:.2f}% MAC_ACC: {:.2f}% MAC_F1: {:.2f}%'.format(metrics[0] * 100,
                                                                                                    metrics[2] * 100,
                                                                                                    metrics[3] * 100,
                                                                                                    metrics[5] * 100))
        print()
    print('[END_OF_TRAINING_STAGE]')
    
model_train()

Some weights of the model checkpoint at ./pre_trained/resnet-152 were not used when initializing ResNetModel: ['classifier.1.bias', 'classifier.1.weight']
- This IS expected if you are initializing ResNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ResNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at ./pre_trained/bert-base-uncased were not used when initializing CatModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.

[START_OF_TRAINING_STAGE]
[EPOCH01][TRAIN] - LOSS:0.922694 ACC_RATE:57.39% WEIGHTED_ACC: 52.60% WEIGHTED_F1: 53.75% MAC_ACC: 42.67% MAC_F1: 38.63%
         [EVAL]  - LOSS:0.840692 ACC_RATE:65.25% WEIGHTED_ACC: 59.57% WEIGHTED_F1: 59.93% MAC_ACC: 44.40% MAC_F1: 39.53%

[EPOCH02][TRAIN] - LOSS:0.772797 ACC_RATE:64.69% WEIGHTED_ACC: 62.33% WEIGHTED_F1: 62.36% MAC_ACC: 56.56% MAC_F1: 50.65%
         [EVAL]  - LOSS:0.842424 ACC_RATE:66.25% WEIGHTED_ACC: 60.73% WEIGHTED_F1: 61.66% MAC_ACC: 45.86% MAC_F1: 41.51%

[EPOCH03][TRAIN] - LOSS:0.702182 ACC_RATE:69.03% WEIGHTED_ACC: 67.69% WEIGHTED_F1: 67.50% MAC_ACC: 63.67% MAC_F1: 57.41%
         [EVAL]  - LOSS:0.815923 ACC_RATE:64.75% WEIGHTED_ACC: 62.97% WEIGHTED_F1: 63.12% MAC_ACC: 49.08% MAC_F1: 46.91%

[EPOCH04][TRAIN] - LOSS:0.593567 ACC_RATE:75.94% WEIGHTED_ACC: 75.23% WEIGHTED_F1: 74.87% MAC_ACC: 72.80% MAC_F1: 66.70%
         [EVAL]  - LOSS:0.838047 ACC_RATE:63.50% WEIGHTED_ACC: 59.91% WEIGHTED_F1: 59.59% MAC_ACC: 46.33% MAC_F1: 41.79%

[E

In [2]:
import torch
import numpy as np
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss

from util.data_preprocess import get_data_loader, calc_metrics
from model.CatModel import CatModel
from model.AddModel import AddModel
from model.AttentionCatModel import AttentionCatModel
from model.AttentionAddModel import AttentionAddModel

import argparse

class Arg:
    def __init__(self):
        self.lr = 3e-5
        self.epochs = 5
        self.weight_decay = 1e-5
        self.text_only = True
        self.image_only = False
args = Arg()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

torch.manual_seed(1)
torch.cuda.manual_seed(1)
torch.cuda.manual_seed_all(1)
np.random.seed(1)
"""在种子不变的情况下保证结果一致"""
torch.backends.cudnn.deterministic = True

def model_train():
    """训练模型并保存至./model.pth"""

    train_data_loader, valid_data_loader, test_data_loader = get_data_loader()
    model = CatModel.from_pretrained('./pre_trained/bert-base-uncased')
    model.to(device)
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': args.weight_decay},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    optimizer = AdamW(lr=args.lr, params=optimizer_grouped_parameters)
    criterion = CrossEntropyLoss()
    best_rate = 0
    print('[START_OF_TRAINING_STAGE]')
    for epoch in range(args.epochs):
        total_loss = 0
        correct = 0
        total = 0
        target_list = []
        pred_list = []
        model.train()
        for idx, (guid, tag, image, text) in enumerate(train_data_loader):
            tag = tag.to(device)
            image = image.to(device)
            text = text.to(device)
            if args.text_only:
                out = model(image_input=None, text_input=text)
            elif args.image_only:
                out = model(image_input=image, text_input=None)
            else:
                out = model(image_input=image, text_input=text)
            loss = criterion(out, tag)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            total_loss += loss.item() * len(guid)
            pred = torch.max(out, 1)[1]
            total += len(guid)
            correct += (pred == tag).sum()

            target_list.extend(tag.cpu().tolist())
            pred_list.extend(pred.cpu().tolist())

        total_loss /= total
        print('[EPOCH{:02d}]'.format(epoch + 1), end='')
        print('[TRAIN] - LOSS:{:.6f}'.format(total_loss), end='')
        rate = correct / total * 100
        print(' ACC_RATE:{:.2f}%'.format(rate), end='')
        metrics = calc_metrics(target_list, pred_list)
        print(' WEIGHTED_ACC: {:.2f}% WEIGHTED_F1: {:.2f}% MAC_ACC: {:.2f}% MAC_F1: {:.2f}%'.format(metrics[0] * 100,
                                                                                                    metrics[2] * 100,
                                                                                                    metrics[3] * 100,
                                                                                                    metrics[5] * 100))

        total_loss = 0
        correct = 0
        total = 0
        target_list = []
        pred_list = []
        model.eval()

        for guid, tag, image, text in valid_data_loader:
            tag = tag.to(device)
            image = image.to(device)
            text = text.to(device)

            if args.text_only:
                out = model(image_input=None, text_input=text)
            elif args.image_only:
                out = model(image_input=image, text_input=None)
            else:
                out = model(image_input=image, text_input=text)

            loss = criterion(out, tag)

            total_loss += loss.item() * len(guid)
            pred = torch.max(out, 1)[1]
            total += len(guid)
            correct += (pred == tag).sum()

            target_list.extend(tag.cpu().tolist())
            pred_list.extend(pred.cpu().tolist())

        total_loss /= total
        print('         [EVAL]  - LOSS:{:.6f}'.format(total_loss), end='')
        rate = correct / total * 100
        print(' ACC_RATE:{:.2f}%'.format(rate), end='')
        metrics = calc_metrics(target_list, pred_list)
        print(' WEIGHTED_ACC: {:.2f}% WEIGHTED_F1: {:.2f}% MAC_ACC: {:.2f}% MAC_F1: {:.2f}%'.format(metrics[0] * 100,
                                                                                                    metrics[2] * 100,
                                                                                                    metrics[3] * 100,
                                                                                                    metrics[5] * 100))
        print()
    print('[END_OF_TRAINING_STAGE]')
    
model_train()

Some weights of the model checkpoint at ./pre_trained/resnet-152 were not used when initializing ResNetModel: ['classifier.1.bias', 'classifier.1.weight']
- This IS expected if you are initializing ResNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ResNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at ./pre_trained/bert-base-uncased were not used when initializing CatModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.

[START_OF_TRAINING_STAGE]
[EPOCH01][TRAIN] - LOSS:0.788448 ACC_RATE:65.33% WEIGHTED_ACC: 63.32% WEIGHTED_F1: 61.21% MAC_ACC: 60.01% MAC_F1: 45.46%
         [EVAL]  - LOSS:0.666586 ACC_RATE:74.00% WEIGHTED_ACC: 72.75% WEIGHTED_F1: 71.28% MAC_ACC: 69.25% MAC_F1: 55.50%

[EPOCH02][TRAIN] - LOSS:0.566357 ACC_RATE:76.83% WEIGHTED_ACC: 75.78% WEIGHTED_F1: 75.27% MAC_ACC: 71.90% MAC_F1: 64.00%
         [EVAL]  - LOSS:0.665503 ACC_RATE:74.75% WEIGHTED_ACC: 75.01% WEIGHTED_F1: 72.45% MAC_ACC: 76.09% MAC_F1: 57.29%

[EPOCH03][TRAIN] - LOSS:0.337518 ACC_RATE:86.81% WEIGHTED_ACC: 86.42% WEIGHTED_F1: 86.45% MAC_ACC: 82.89% MAC_F1: 80.10%
         [EVAL]  - LOSS:0.734645 ACC_RATE:72.25% WEIGHTED_ACC: 68.27% WEIGHTED_F1: 69.43% MAC_ACC: 54.36% MAC_F1: 50.27%

[EPOCH04][TRAIN] - LOSS:0.177464 ACC_RATE:93.94% WEIGHTED_ACC: 93.90% WEIGHTED_F1: 93.91% MAC_ACC: 92.31% MAC_F1: 91.62%
         [EVAL]  - LOSS:0.887878 ACC_RATE:73.00% WEIGHTED_ACC: 71.90% WEIGHTED_F1: 72.30% MAC_ACC: 61.80% MAC_F1: 59.93%

[E

In [19]:
import pandas as pd
import numpy as np
add_result = pd.read_csv('./output/add_model_test_with_label.txt')
attention_add_result = pd.read_csv('./output/attention_add_model_test_with_label.txt')
cat_result = pd.read_csv('./output/cat_model_test_with_label.txt')
attention_cat_result = pd.read_csv('./output/attention_cat_model_test_with_label.txt')

In [23]:
add_result

,guid,tag
0,8,positive
1,1576,negative
2,2320,positive
3,4912,negative
4,3821,positive
...,...,...
506,1048,positive
507,1059,positive
508,1485,positive
509,3195,negative


In [21]:
result = pd.DataFrame()
result['guid'] = add_result['guid']
result['tag1'] = add_result['tag']
result['tag2'] = attention_add_result['tag']
result['tag3'] = cat_result['tag']
result['tag4'] = attention_cat_result['tag']
tags = []
for row in result.values:
    guid = row[0]
    count = {
        'positive': np.count_nonzero(row == 'positive'),
        'negative': np.count_nonzero(row == 'negative'),
        'neutral': np.count_nonzero(row == 'neutral')
    }
    max_count = 0
    max_count_tag = 'positive'
    two_count = 0
    for k, v in count.items():
        if v == 2:
            two_count += 1
        if v > max_count:
            max_count = v
            max_count_tag = k
    tag = None
    if max_count > 2 or two_count == 1:
        tag = max_count_tag
    else:
        if count['neutral'] == 2:
            if count['positive'] == 2:
                tag = 'positive'
            else:
                tag == 'negative'
        else:
            tag = 'neutral'
    tags.append(tag)
result['tag'] = tags
result[['guid', 'tag']].to_csv('result.txt')

In [26]:
result[['guid', 'tag']].to_csv('result.txt', index=False)